<a href="https://colab.research.google.com/github/carlcodecode/Data-Science-II/blob/main/SBERTSearchDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util

# Load Dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [2]:
# Load sbert model
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
from sentence_transformers import InputExample
import random

# Prepare Training Data

positive_review = [review["text"] for review in dataset["train"] if review["label"] == 1]
negative_review = [review["text"] for review in dataset["train"] if review["label"] == 0]

train_examples = []

for i in range(1000):
  #random pairing strategy

  #pos-pos pair
  a, b = random.sample(positive_review, 2)
  train_examples.append(InputExample(texts=[a,b], label = 1.0))
  #neg-neg pair
  a, b = random.sample(negative_review, 2)
  train_examples.append(InputExample(texts=[a,b], label = 1.0))
  #pos-neg pair
  a = random.choice(positive_review)
  b = random.choice(negative_review)

  train_examples.append(InputExample(texts=[a,b], label=0.0)) # not similar

In [4]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128

train_loader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)

In [5]:
from sentence_transformers import losses

# Choose Loss

loss_fn = losses.CosineSimilarityLoss(model) # use cosine similarity to find distance between pairs of sentences

In [6]:
# Fine Tune Step

model.fit(
    train_objectives=[(train_loader, loss_fn)],
    epochs = 3,
    warmup_steps = int(0.1*len(train_loader) * 3),
    show_progress_bar = True
)

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: carlaguinaldo (carlaguinaldo-university-of-houston) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [7]:
# saving the model

model.save("output/sbert-imdb")

In [8]:
# Reload model

search_model = SentenceTransformer("output/sbert-imdb")

In [9]:
# build embeddings

corpus = dataset["train"]

embeddings = search_model.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

In [10]:
# simple search function

from sentence_transformers import util
import torch

def sbert_search(query, top_k=5):
  query_embedding = search_model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(query_embedding, embeddings, top_k=top_k)[0]
  return [(hit["corpus_id"], hit["score"], corpus[hit["corpus_id"]]) for hit in hits]



In [12]:
# search

query = "Story bad"
results =  sbert_search(query, 10)

print(f"Top 10 results:")
for i, (_, score, text) in enumerate(results):
  print(f"{i+1}: {text}, with score: {score:.2f}")

Top 10 results:
1: {'text': "This thing was bad. Really bad. I mean, low budget can sometimes be very inspiring, but not this. The story was so off-the-shelf, the alien's behaviour so illogical, the characters so clichéed. I found nothing good in it. And I did try.", 'label': 0}, with score: 0.59
2: {'text': 'This isn\'t a bad movie... but it\'s the type your girlfriend makes you watch. The story isn\'t bad .... it just makes the hour and a half seem so long. It\'s hard for me to trash this movie because I really do like the idea of it but it was just to long and thin on story... too bad. The main character never really seems to change all that much from beginning to end . I mean goes through something "life changing" and he can barely break a smile. I really thought Mac was really good. His weird quirkiness kept the movie from being a complete disaster. Maybe I just don\'t "get" this film. If anyone can explain it to me I\'d love to be informed.', 'label': 0}, with score: 0.56
3: {'te